In [ ]:
import requests
from pprint import pprint

In [ ]:
response = requests.get(
    "https://ws.audioscrobbler.com/2.0/",
    params={
        "method": "user.gettoptracks",
        "user": "cocacooler",
        "api_key": "ae26d4892e373c6fc188acf7e3cb36c3",
        "page": "1",
        "format": "json"
    }
)

In [ ]:
API_KEY="ae26d4892e373c6fc188acf7e3cb36c3"
USERNAME="cocacooler"

In [ ]:
def request_page(api_key, username, page):
    response = requests.get(
        "https://ws.audioscrobbler.com/2.0/",
        params={
            "method": "user.gettoptracks",
            "user": username,
            "api_key": api_key,
            "page": str(page),
            "format": "json"
        }
    )
    assert response.status_code == 200
    return response

In [ ]:
def extract_tracks(page_json):
    return [
        {
            "artist": track["artist"]["name"],
            "title": track["name"],
            "lastfm_url": track["url"],
            "musicbrainz_id": track["mbid"]
            
        }
        for track in page_json["toptracks"]["track"]]

In [ ]:
def get_tracks(api_key=API_KEY, username=USERNAME, num_tracks=100):
    tracks = []
    page1_json = request_page(api_key, username, 1).json()
    
    total_tracks = int(page1_json["toptracks"]["@attr"]["total"])
    assert total_tracks >= num_tracks
    
    tracks_per_page = int(page1_json["toptracks"]["@attr"]["perPage"])
    
    # need_pages * tracks_per_page >= num_tracks
    need_pages = num_tracks // tracks_per_page + 1
    
    tracks.extend(extract_tracks(page1_json))
    
    for page in range(2, need_pages + 1):
        page_json = request_page(api_key, username, page).json()
        tracks.extend(extract_tracks(page_json))
        
    return tracks

In [ ]:
get_tracks()